In [38]:
import numpy as np
from scipy import optimize
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import math
import seaborn as sns
sns.set()

import scipy.optimize

### Modelo 1

In [9]:
def π_get(price, up, down, r_f):
    return np.round(scipy.optimize.newton(lambda π: (π*up + (1-π)*down) / (1 + r_f) - price, 0.5), 4)
def price_get(π, up, down, r_f):
    return np.round((π*up + (1-π)*down) / (1 + r_f), 4)

In [190]:
# A.65.a (41)
costo_proyecto = 104
π = π_get(100, 180, 60, 0.08)
A = price_get(π, 324, 108, 0.08)
B = price_get(π, 108, 36, 0.08)
price = price_get(π, A, B, 0.08)
print('π: ', π)
print('p1=>',str(p1),' p2=>',str(np.round(p2,2)),'p3:',str(p3))
print('Price equity: ', price)

valor_proyecto = price - costo_proyecto
print(valor_proyecto)

π:  0.4
p1=> 275.6  p2=> 59.6 p3: 0
Price equity:  100.0
-4.0


In [191]:
# A.65.b (41)
π = π_get(100, 180, 60, 0.08) # => volatilidad alta: 100=>[60,180]
debt = 40
debt_notional = debt * 1.1 ** 2

# aca gana el equity: al aumentar 'debt', el proyecto le es menos costoso al equity
# esto es parcialmente compensado por el hecho que debt_notional va a ser mayor. Pero será lo suficientemente mayor? 
# cómo pricearlo? (por ahora, método simplista, en el cuál el equity se morfa a la deuda)
# la deuda se aviva de hacer un FAIR VALUATION?
# => Hay rational expectations aca? NO. No se aviva de las implicancias del riesgo del proyecto
costo_para_equity = costo_proyecto - debt

p1 = max(324 - debt_notional, 0)
p2 = max(108 - debt_notional, 0)
p3 = max(36 - debt_notional, 0)
A = price_get(π, p1, p2, 0.08)
B = price_get(π, p2, p3, 0.08)
price = price_get(π, A, B, 0.08)
print('π: ', π)
print('p1: ',str(p1),' p2: ',str(np.round(p2,2)),'p3: ',str(p3))
print('Price equity: ', price)

valor_para_equity = price - costo_para_equity
print('Valor p/ equity: ', np.round(valor_para_equity, 4))
print('Valor de financial flexibility: ', np.round(valor_para_equity - valor_proyecto, 4))

π:  0.4
p1:  275.6  p2:  59.6 p3:  0
Price equity:  62.332
Valor p/ equity:  -1.668
Valor de financial flexibility:  2.332


In [193]:
# A.41.c
π = π_get(100, 180, 60, 0.08)
debt = 60
debt_notional = debt * 1.1 ** 2 # => costo de la deuda cte. (la deuda no se está avivando)
costo_para_equity = costo_proyecto - debt

p1 = max(324 - debt_notional, 0)
p2 = max(108 - debt_notional, 0)
p3 = max(36 - debt_notional, 0)
A = price_get(π, p1, p2, 0.08)
B = price_get(π, p2, p3, 0.08)
price = price_get(π, A, B, 0.08)
print('π: ', π)
print('p1: ',str(np.round(p1,2)),' p2: ',str(np.round(p2,2)),'p3: ',str(p3))
print('Price equity: ', price)

valor_para_equity = price - costo_para_equity
print('Valor p/ equity: ', np.round(valor_para_equity, 4))
print('Valor de financial flexibility: ', np.round(valor_para_equity - valor_proyecto, 4))

π:  0.4
p1:  251.4  p2:  35.4 p3:  0
Price equity:  49.0535
Valor p/ equity:  5.0535
Valor de financial flexibility:  9.0535


In [ ]:
# Conclusión: si una opción real aumenta el valor a un proyecto, es porque en algún lado hay una asimetría.
# => le transferís la negatividad del proyecto a otro jugador
# => le pasas el downside risk a la deuda y te quedas con el upside
# => 

### Modelo 2

In [197]:
# [75/88] Equity vs. Debt
# NOTA: Todo esto es a un año. Se podría generalizar a N años
# Suponemos que la cía es twin de un sintético (para usar martingalas)
# Suponemos total racionalidad, informacion completa, equilibrio general
# Deuda reacciona ante leverage: deuda fairly valued: según el nivel de financiación, la deuda es endógena, al aumentar leverage
# se pricea (se lleva la esperanza matemática justa)
# Exógeno: retorno esperado del mercado s&p (r_P), 
# Endógeno: principal de la deuda, 
r_P = 0.07
up = 110
down = 40
r_f = 0.05
π = 0.5

# NPV (equity + deuda)
valor_de_mercado = np.round((π * up + π * down) / (1 + r_P), 3)

r_up = up / valor_de_mercado - 1
r_down = down / valor_de_mercado - 1
r_expected = np.round(π * r_up + π * r_down, 3)
r_variance = π * (r_up - r_expected)**2 + π * (r_down - r_expected)**2
print('r_variance: ', str(r_variance))

percentage_equity = 0.2
percentage_debt = 1 - percentage_equity

# x = nocional de la deuda
# vhd = valor spot (hoy) de la deuda
vhd = percentage_debt * valor_de_mercado

# principal_debt: nocional en maturity (enogeneiza el nocional)
# Solver: debe suceder que vhd = esperanza del principal de la deuda (utilizando x como endógena)
principal_debt = scipy.optimize.newton(lambda x: (π * min(up, x) + π * min(down, x)) / (1+r_f) - vhd, 0)
print('Valor spot deuda: ', vhd)
print('Valor en maturity deuda: ', principal_debt)

yield_debt = principal_debt / vhd - 1
# Yield de la deuda: es ENDGÓGENO: el equity no lo controla.
print('Yield debt: ', yield_debt)

# Valor hoy del equity: Exógeno
vhe = percentage_equity * valor_de_mercado
principal_equity = π * max(0, up-principal_debt) + π * max(0, down-principal_debt) # CF del equity a 1 año (x esperanza matem.)

# Yield del equity: Endógeno (viene del principal_debt)
r_e_expected = principal_equity / vhe - 1
print('Expected return equity: ', r_e_expected)

# Valor hoy equity
print('Valor hoy equity: ', vhe)

# Volatilidad retornos equity
r1 = max(0, up-principal_debt) / vhe - 1
r2 = max(0, (down-principal_debt)) / vhe - 1
r3 = np.round(π * r1 + π * r2, 3)
r4 = π * (r1 - r3)**2 + π * (r2 - r3)**2
# print(r1, r2, r3)
print('Volatilidad retornos equity: ', math.sqrt(r4))

# Valor hoy deuda
print('Valor hoy deuda: ', vhd)

# Volatilidad retornos deuda
r1 = min(up, principal_debt) / vhd - 1
r2 = min(down, principal_debt) / vhd - 1
r3 = np.round(π * r1 + π * r2, 3)
r4 = π * (r1 - r3)**2 + π * (r2 - r3)**2
print(r1, r2, r3)
print('Volatilidad retornos deuda: ', math.sqrt(r4))

r_variance:  0.2493370358199391
Valor spot deuda:  56.074400000000004
Valor en maturity deuda:  77.75624000000002
Yield debt:  0.38666200619177404
Expected return equity:  0.15003495356169583
Valor hoy equity:  14.018600000000001
Volatilidad retornos equity:  1.150034954092876
Valor hoy deuda:  56.074400000000004
0.38666200619177404 -0.28666200619177384 0.05
Volatilidad retornos deuda:  0.33666200619177394


In [137]:
# [80/160] Equity vs. Debt (continuación)
# IDEA: Variar la estructura del capital (Equity / Debt)
r_P = 0.05
up = 110
down = 110
r_f = 0.05 # => como up==down, debe ser r_P == r_f
π = 0.5

# NPV (equity + deuda)
valor_de_mercado = np.round((π * up + π * down) / (1 + r_P), 3)

r_up = up / valor_de_mercado - 1
r_down = down / valor_de_mercado - 1
r_expected = np.round(π * r_up + π * r_down, 3)
r_variance = π * (r_up - r_expected)**2 + π * (r_down - r_expected)**2

percentage_equity = 0.2
percentage_debt = 1 - percentage_equity

# x = nocional de la deuda
# vhd = valor spot (hoy) de la deuda
vhd = percentage_debt * valor_de_mercado

# principal_debt: nocional en maturity (enogeneiza el nocional)
# Solver: debe suceder que vhd = esperanza del principal de la deuda (utilizando x como endógena)
principal_debt = scipy.optimize.newton(lambda x: (π * min(up, x) + π * min(down, x)) / (1+r_f) - vhd, 0)
print('Valor spot deuda: ', vhd)
print('Valor en maturity deuda: ', principal_debt)

yield_debt = principal_debt / vhd - 1
# Yield de la deuda: es ENDGÓGENO: el equity no lo controla.
print('Yield debt: ', yield_debt)

# Valor hoy del equity: Exógeno
vhe = percentage_equity * valor_de_mercado
principal_equity = π * max(0, up-principal_debt) + π * max(0, down-principal_debt) # CF del equity a 1 año (x esperanza matem.)

# Yield del equity: Endógeno (viene del principal_debt)
r_e_expected = principal_equity / vhe - 1
print('Expected return equity: ', r_e_expected)

# Valor hoy equity
print('Valor hoy equity: ', vhe)

# Volatilidad retornos equity
r1 = max(0, up-principal_debt) / vhe - 1
r2 = max(0, (down-principal_debt)) / vhe - 1
r3 = np.round(π * r1 + π * r2, 3)
r4 = π * (r1 - r3)**2 + π * (r2 - r3)**2
print('Volatilidad retornos equity: ', math.sqrt(r4))

# Valor hoy deuda
print('Valor hoy deuda: ', vhd)

# Volatilidad retornos deuda
r1 = min(up, principal_debt) / vhd - 1
r2 = min(down, principal_debt) / vhd - 1
r3 = np.round(π * r1 + π * r2, 3)
r4 = π * (r1 - r3)**2 + π * (r2 - r3)**2
print('Volatilidad retornos deuda: ', math.sqrt(r4))

Valor spot deuda:  83.8096
Valor en maturity deuda:  88.00008000000001
Yield debt:  0.050000000000000044
Expected return equity:  0.049995227277065624
Valor hoy equity:  20.9524
Volatilidad retornos equity:  4.772722934379137e-06
Valor hoy deuda:  83.8096
Volatilidad retornos deuda:  4.163336342344337e-17


In [ ]:
# [80-90/160] NOTAS
# principal de la deuda < down node (deuda libre de riesgo?)
# equity no se lleva lo mismo en ambos escenarios. Equity es más riesgoso que la deuda
# a mayor debt/equity, el retorno esperado del equity crece (mayor palanca) para el riesgo, deuda siempre es libre de riesgo (dependiend up/down nodes)

# Si la deuda es defaulteable (no libre de riesgo), el yield de la deuda va a subir. En el grafico de f(debt/equity) = ret_equity,
# hay un punto en el cual la deuda empieza a pedir cada vez más, hasta que en un momento pide más yield que el equity.

# Conclusión: NO SIEMPRE PASA QUE ANTE AUMENTO EN EL APALANCAMIENTO GANA EL EQUITY. HAY QUE VER CUÁL ES EL MODELO
# => en estos modelos: DADOS LOS PARÁMETROS, PUEDO MEDIR EL RIESGO DE DEFAULT

In [182]:
# [76/88] 
# Hay una empresa con equity y deuda. Viene un nuevo proyecto. A quién le conviene qué tipo de proyecto? Riesgoso o no riesgoso?
# variable endógena => fairly valued (lo podés calcular). A nivel esperanza matemática te cobré todo.
# deuda_original = y; deuda_nueva = x => %_nueva_deuda = x/(x+y) => se repartirán par y paso?
# f = x / (x+y)
# f_obj: si hay plata te llevas x y sino, a prorata

In [183]:
# [46/160]
# 